In [2]:
from typing import Literal, TypedDict, Union

import os
import requests
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3 import Web3
from web3.middleware import SignAndSendRawMiddlewareBuilder

from hyperliquid.utils import constants
from hyperliquid.utils.signing import get_timestamp_ms, sign_l1_action

from dotenv import load_dotenv

# take environment variables
load_dotenv()

class CreateInputParams(TypedDict):
    nonce: int


class CreateInput(TypedDict):
    create: CreateInputParams


FinalizeEvmContractInput = Union[Literal["firstStorageSlot"], CreateInput]


class FinalizeEvmContractAction(TypedDict):
    type: Literal["finalizeEvmContract"]
    token: int
    input: FinalizeEvmContractInput


SHOULD_DEPLOY_CONTRACT = False  # change this if you are happy with your deployed contract and want to skip this
SHOULD_LINK_CONTRACT = True  # change this to True if you want to link your token, this process is not reversible!

DEFAULT_CONTRACT_ADDRESS = Web3.to_checksum_address(
    os.getenv("TOKEN_CONTRACT_ADDRESS")  # change this to your contract address if you are skipping deploying
)
TOKEN = int(os.getenv("TOKEN_ID"))  # note that if changing this you likely should also change the abi to have a different name and perhaps also different decimals and initial supply
PRIVATE_KEY = os.getenv("PRIVATE_KEY")  # Change this to your private key

# Connect to the JSON-RPC endpoint
rpc_url = "https://rpc.hyperliquid-testnet.xyz/evm"
w3 = Web3(Web3.HTTPProvider(rpc_url))

# The account will be used both for deploying the ERC20 contract and linking it to your native spot asset
# You can also switch this to create an account a different way if you don't want to include a secret key in code
if PRIVATE_KEY == "0xPRIVATE_KEY":
    raise Exception("must set private key or create account another way")
account: LocalAccount = Account.from_key(PRIVATE_KEY)
print(f"Running with address {account.address}")
w3.middleware_onion.add(SignAndSendRawMiddlewareBuilder.build(account))
w3.eth.default_account = account.address
# Verify connection
if not w3.is_connected():
    raise Exception("Failed to connect to the Ethereum network")

creation_nonce: int
contract_address = DEFAULT_CONTRACT_ADDRESS
creation_nonce = int(os.getenv("CREATION_NONCE"))

if SHOULD_LINK_CONTRACT:
    if contract_address is None:
        raise Exception("contract address cannot be None")
    action = {
        "type": "spotDeploy",
        "requestEvmContract": {
            "token": TOKEN,
            "address": contract_address.lower(),
            "evmExtraWeiDecimals": 13,
        },
    }
    nonce = get_timestamp_ms()
    signature = sign_l1_action(account, action, None, nonce, None, False)
    payload = {
        "action": action,
        "nonce": nonce,
        "signature": signature,
        "vaultAddress": None,
    }

    print("First post request")
    print(f"Account: {account.address}")
    print(f"Token: {TOKEN}")
    print(f"Address: {contract_address}")
    print(f"Signature: {signature}")
    print(f"Payload: {payload}")
    print(f"Linking contract to token {TOKEN} at address {contract_address}...")

    response = requests.post(constants.TESTNET_API_URL + "/exchange", json=payload, timeout=10)
    print(response.json())

    use_create_finalization = True
    finalize_action: FinalizeEvmContractAction
    if use_create_finalization:
        finalize_action = {
            "type": "finalizeEvmContract",
            "token": TOKEN,
            "input": {"create": {"nonce": creation_nonce}},
        }
    else:
        finalize_action = {"type": "finalizeEvmContract", "token": TOKEN, "input": "firstStorageSlot"}
    nonce = get_timestamp_ms()
    signature = sign_l1_action(account, finalize_action, None, nonce, None, False)
    payload = {
        "action": finalize_action,
        "nonce": nonce,
        "signature": signature,
        "vaultAddress": None,
    }

    print()
    print("Second post request")
    print(f"Account: {account.address}")
    print(f"Token: {TOKEN}")
    print(f"Address: {contract_address}")
    print(f"Signature: {signature}")
    print(f"Payload: {payload}")
    print(
        f"Finalizing linking contract to token {TOKEN} at address {contract_address}..."
    )
    
    response = requests.post(constants.TESTNET_API_URL + "/exchange", json=payload, timeout=10)
    print(response.json())

Running with address 0xcCe5E9853868e2C6603e577BcB2D51356f3b2f53
First post request
Account: 0xcCe5E9853868e2C6603e577BcB2D51356f3b2f53
Token: 1299
Address: 0x3FB075C681e4a178777476af44eE9b7ef4186B11
Signature: {'r': '0x1a4cb014d45efa3593913d819bde833fb7b069b791a516cdcb5faba767d40e57', 's': '0x5e4c145bc18a631d0e11fa0aa07f29569ce566c756dbf12212479b93053c1574', 'v': 28}
Payload: {'action': {'type': 'spotDeploy', 'requestEvmContract': {'token': 1299, 'address': '0x3fb075c681e4a178777476af44ee9b7ef4186b11', 'evmExtraWeiDecimals': 13}}, 'nonce': 1746025925265, 'signature': {'r': '0x1a4cb014d45efa3593913d819bde833fb7b069b791a516cdcb5faba767d40e57', 's': '0x5e4c145bc18a631d0e11fa0aa07f29569ce566c756dbf12212479b93053c1574', 'v': 28}, 'vaultAddress': None}
Linking contract to token 1299 at address 0x3FB075C681e4a178777476af44eE9b7ef4186B11...
{'status': 'ok', 'response': {'type': 'default'}}

Second post request
Account: 0xcCe5E9853868e2C6603e577BcB2D51356f3b2f53
Token: 1299
Address: 0x3FB075C68